In [18]:
import requests
import yfinance as yf
import bs4 as bs
from collections import defaultdict
import pandas as pd
import datetime

In [19]:
#Creating the user variables
route = "C:/Users/usuario/MovesRecordt.txt" #DB route
wallet = {"user":"name",
          "money":0,
          "stocks": defaultdict(int)
         }
wallet["user"]="Esteban"
print(wallet)
#Initialize variables with previous data from the "DataBase" (A txt document)

{'user': 'Esteban', 'money': 0, 'stocks': defaultdict(<class 'int'>, {})}


In [20]:
def readprev():
    with open(route, "r") as document:
        for line in document:
            #Recovering the money balance
            if line[:10]=="MONEY SAVE":
                end = line.find("U")
                num = float(line[13:end])
                wallet["money"] = num
            
            #Recovering the shares per company
            elif line[:12] == "COMPANY SAVE":
                #Finding company name
                start_index = 24
                end_index = start_index
                finded = False
                while not finded:
                    end_index += 1 
                    if line[end_index]==" ":
                        finded = True
                        break
                company = line[start_index:end_index]
                
                #Finding shares amount
                index1 = line.find("Amount: ")+8
                amount = int(line[index1:])
                
                #Loading data into the initial user variables
                wallet["stocks"][company]=amount

In [21]:
def get_company_name(ticker):
    company=yf.Ticker(ticker)
    company_info=company.info
    info=company_info.get("longBusinessSummary")
    return info[:info.find(".")]

In [22]:
def save_before_close():
    #Saves the amount of money before closing the app
    money_amount = wallet["money"]
    register1 = f"MONEY SAVE | {money_amount}USD \n"
    with open(route, "a") as doc:
            doc.write(register1)
    
    #Saves the actual amunt of shares before exiting
    for item in wallet["stocks"].items():
        company = item[0]
        amount = item[1]
        register2 = f"COMPANY SAVE | Company: {company} | Shares Amount: {amount} \n"
        with open(route, "a") as doc:
            doc.write(register2)
        

In [23]:
#Writing in a txt the moves of the program,using it as a database

#Function to write buy/sell movements types
def save_buy_sell(action,money_amount,share_amount,ticker):
    company = get_company_name(ticker)
    
    now = datetime.datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")
              
    register = f"{action} | {company} | {share_amount} | {money_amount}USD | {date} | {time}\n"
    with open(route, "a") as doc:
        doc.write(register)

#Function to write deposit movements types
def save_deposit(money_amount):
    now = datetime.datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")
    
    register = f"DEPOSIT | {money_amount}USD | {date} | {time}\n"
    with open(route, "a") as doc:
        doc.write(register)

In [24]:
#Printing menu:
def menu_input():
    print("-----------------------------------------")
    print("Actions:")
    print("1-Ask All stock prices from Nasdaq 100")
    print("2-Ask a specific stock price")
    print("3-Buy a stock")
    print("4-Sell a stock")
    print("5-Deposit Money")
    print("6-See Operations History")
    print("7-Check Balance")
    print("8-Check global profitability")
    print("9-Check profitabilty per stock")
    print("10-Close Program")
    print("-----------------------------------------")
    action = input("Please enter an action: ")
    if action.isnumeric():
        return int(action)
    else:
        print("Invalid Option \n")
        return None

In [25]:
def get_ticket_price(company_ticket):
    #Getting all company information with a dict
    company=yf.Ticker(company_ticket)
    company_info=company.info
    #Returning the price from that long dict
    return company_info.get("currentPrice")

In [26]:
def deposit_money(amount):
    if amount.isnumeric() and int(amount) > 0:
        wallet["money"]+=int(amount)
        save_deposit(amount)
    else:
        print("Invalid input, please enter a number > 0 \n")   

In [27]:
symbol = []
company = []
prices = []
def ask_all():
    #Getting a list of all the tickets of Nasdaq 100 from Wikipedia
    resp=requests.get("https://en.wikipedia.org/wiki/Nasdaq-100")
    resp=resp.text
    soup = bs.BeautifulSoup(resp,"lxml")
    table=soup.findAll("table",{"class":"wikitable"})[3]

    for row in (table.findAll("tr")[1:]):
        ticket = row.findAll('td')[1].text
        symbol.append(ticket)
        company.append(row.findAll("td")[0].text)
        prices.append(get_ticket_price(ticket))
    
    #Creating a dataframe to show the lists
    df = pd.DataFrame(list(zip(company,symbol,prices)),
                     columns=["Company","Symbol","Current Price"])
    df = df.reset_index(drop=True)
    df = df.set_index("Company")
    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(df)

In [28]:
def buy_stock(ticker,amount):
    #Check money balance
    output=""
    price = get_ticket_price(ticker)*amount
    if price > wallet["money"]:
        output = "Insufficient balance,operation cancelled"
    else:
        wallet["money"]-=price
        wallet["stocks"][ticker]+=amount
        save_buy_sell("BUY",price,amount,ticker)
        output="Operation completed successfully"
    return output

In [29]:
def sell_stock(ticker,amount):
    #Check amount
    if ((wallet["stocks"].__contains__(ticker))==False) or (amount > wallet["stocks"][ticker]):
        print("Insufficient amount of stocks to sell,operation cancelled")
    else:
        price = amount*get_ticket_price(ticker)
        wallet["money"]+= price
        wallet["stocks"][ticker]-= amount
        save_buy_sell("SELL",price,amount,ticker)
        if wallet["stocks"][ticker]==0:
            wallet["stocks"].pop(ticker)
        print("You've sold",amount,"shares of",get_company_name(ticker))

In [30]:
def check_wallet():
    print("\n------------------------------------------------")
    print("The user",wallet["user"],"has: ")
    print(wallet["money"],"USD \n")
    print("And the following stocks:")
    for ticker, amount in wallet["stocks"].items():
        print(ticker,"-->",amount)
    print("------------------------------------------------")

In [31]:
def history():
    with open(route,"r") as document:
        for line in document:
            operation = line[:line.find("|")-1]
            if operation == "BUY" or operation=="SELL" or operation=="DEPOSIT":
                print(line)

In [32]:
def global_profit():
    global_value = wallet["money"]
    for company in wallet["stocks"]:
        global_value += get_ticket_price(company)*wallet["stocks"][company]
        
    initial_value = 0
    with open(route,"r") as document:
        for line in document:
              if line[:7]=="DEPOSIT":
                    initial_value += float(line[10:line.find("U")])
                    
    profit = round(((global_value/initial_value)-1)*100,2)
    print("Your profit is: ",profit,"%")

In [ ]:
#Main Loop
print("Welcome to Investing Simulator")
readprev()
while True:
    action = menu_input()
    if action==1:
        print("Please wait, this can take a while, some prices may not be available...")
        ask_all()
    elif action==2:
        company = input("Please enter the company ticket you want to check: ")
        price = get_ticket_price(company)
        print("\n The price per share of your company is $",price,"\n")
    elif action==3:
        ticker = input("Enter the ticker of  the company you want to buy: ")
        amount = input("How many units? ")
        if amount.isdigit(): 
            print(buy_stock(ticker,int(amount)))
        else:
            print("Please enter a integer number for the amount.")
    elif action==4:
        #Sell stock
        ticker = input("Enter the ticker of the compay you'd like to sell: ")
        amount = int(input("Enter the amount of shares to sell: "))
        sell_stock(ticker,amount)
    elif action==5:
        amount=input("Please enter an amount of money to deposit in your account: ")#
        deposit_money(amount)
        print("Now, the current balance is:",wallet["money"],"\n")
    elif action==6:
        history()
    elif action==7:
        check_wallet()
    elif action==8:
        global_profit()
    elif action==9:
        print("Future code")
    elif action==10:
        save_before_close()
        print("Execution finished")
        break

Welcome to Investing Simulator
-----------------------------------------
Actions:
1-Ask All stock prices from Nasdaq 100
2-Ask a specific stock price
3-Buy a stock
4-Sell a stock
5-Deposit Money
6-See Operations History
7-Check Balance
8-Check global profitability
9-Check profitabilty per stock
10-Close Program
-----------------------------------------


Please enter an action:  1


Please wait, this can take a while, some prices may not be available...
                               Symbol  Current Price
Company                                             
Adobe Inc.                       ADBE        552.960
Advanced Micro Devices            AMD        138.590
Airbnb                           ABNB        136.920
Alphabet Inc. (Class A)         GOOGL        174.710
Alphabet Inc. (Class C)          GOOG        176.490
Amazon                           AMZN        227.030
American Electric Power           AEP         95.850
Amgen                            AMGN        272.580
Analog Devices                    ADI        218.200
Ansys                            ANSS        345.300
Apple Inc.                       AAPL        242.840
Applied Materials                AMAT        173.020
AppLovin                          APP        401.500
Arm Holdings                      ARM        140.890
ASML Holding                     ASML        708.980
AstraZeneca                

Please enter an action:  2
Please enter the company ticket you want to check:  MELI



 The price per share of your company is $ 1947.18 

-----------------------------------------
Actions:
1-Ask All stock prices from Nasdaq 100
2-Ask a specific stock price
3-Buy a stock
4-Sell a stock
5-Deposit Money
6-See Operations History
7-Check Balance
8-Check global profitability
9-Check profitabilty per stock
10-Close Program
-----------------------------------------


Please enter an action:  7



------------------------------------------------
The user Esteban has: 
6257.320000000001 USD 

And the following stocks:
AMD --> 10
AAPL --> 16
TSLA --> 14
MELI --> 4
ZS --> 10
------------------------------------------------
-----------------------------------------
Actions:
1-Ask All stock prices from Nasdaq 100
2-Ask a specific stock price
3-Buy a stock
4-Sell a stock
5-Deposit Money
6-See Operations History
7-Check Balance
8-Check global profitability
9-Check profitabilty per stock
10-Close Program
-----------------------------------------
